In [ ]:
# Import dependencies

import requests
import pandas as pd
import os
import datetime
from google.cloud import bigquery
import sleep_config

In [ ]:
url = 'https://api.ouraring.com/v2/usercollection/daily_sleep'
url_2 = 'https://api.ouraring.com/v2/usercollection/sleep' 
daily_activity_url = 'https://api.ouraring.com/v2/usercollection/daily_activity'
params={
    'start_date': datetime.datetime.strptime('2022-06-01', '%Y-%m-%d').strftime('%Y-%m-%d'), 
    'end_date': datetime.datetime.now().strftime('%Y-%m-%d')
}
headers = {
    'Authorization': f'Bearer {sleep_config.API_KEY}'
}
response_daily_sleep = requests.request('GET', url, headers=headers, params=params)
response_sleep = requests.request('GET', url_2, headers=headers, params=params)
response_activity = requests.request('GET', daily_activity_url, params=params, headers=headers)
r_daily_sleep = response_daily_sleep.json()
r_sleep = response_sleep.json()
daily_activity = response_activity.json()

In [ ]:
# Convert the Json to a DataFrame
df_daily_sleep_score = pd.json_normalize(r_daily_sleep['data'])
df_sleep = pd.json_normalize(r_sleep['data'])
daily_activity = pd.json_normalize(daily_activity['data'])

In [ ]:
# Rename the columns by replacing "contributors." with an empty string
df_daily_sleep_score.rename(columns=lambda s: s.replace("contributors.", ""), inplace=True)
df_sleep.rename(columns=lambda x: x.split(".")[-1], inplace = True)
daily_activity.rename(columns=lambda s: s.replace("contributors.", ""), inplace=True)

# Convert the day column to a datetime data type
df_daily_sleep_score["day"] = pd.to_datetime(df_daily_sleep_score["day"], format="%Y/%m/%d")
df_sleep["day"] = pd.to_datetime(df_sleep["day"], format="%Y/%m/%d")
daily_activity["day"] = pd.to_datetime(daily_activity["day"], format="%Y/%m/%d")

# Remove the timestamp column
df_daily_sleep_score.drop(columns=['timestamp'], inplace=True)
df_sleep.drop(columns=['average_breath', 'id', 'items', 'type', 'interval', 'sleep_phase_5_min', 'movement_30_sec', 'previous_day_activity', 'previous_night', 'hrv', 'timestamp', 'heart_rate', 'readiness', 'temperature_trend_deviation', 'temperature_deviation'], inplace=True)
daily_activity = daily_activity[['score','total_calories','day', 'steps','equivalent_walking_distance','move_every_hour', 'recovery_time', 'high_activity_time', 'medium_activity_time', 'low_activity_time']]

In [ ]:
# Set up credentials and connect to BigQuery project and load the data from a Pandas DataFrame into a specific table in BigQuery.
client = bigquery.Client()
table_id = sleep_config.TABLE
sleep_table_id = sleep_config.SLEEP_TABLE
daily_activity_table_id = sleep_config.SLEEP_TABLE

job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)
job = client.load_table_from_dataframe(df_daily_sleep_score, table_id, job_config=job_config)
job_sleep = client.load_table_from_dataframe(df_sleep, sleep_table_id, job_config=job_config)
job_activity = client.load_table_from_dataframe(daily_activity, daily_activity_table_id, job_config=job_config)